##### MLFLOW EXPERIMENTATION

In [1]:
import os

print(f"Original Working Directory:{os.getcwd()}")

os.chdir("../")

print(f"Current Working Directory after Changed: {os.getcwd()}")

Original Working Directory:c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\research
Current Working Directory after Changed: c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification


In [2]:
# UPDATE ENTITY FOR MLFLOW EXPERIMENTATION

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelLoggingWithMLFLOWConfig:
    trained_model_path: Path
    all_params: dict
    mlflow_url: str
    scores_file_path: Path



In [3]:
# UPDATE THE CONFIGURATION MANAGER INSIDE THE SRC/CONFIG - THIS IS ESSENTIAL AS THE CURRENT WORKFLOW WILL USE IT TO RUN

from src.Kidney_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

from Kidney_classifier.utils.common import read_yaml_file, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_mlflow_config(self) -> ModelLoggingWithMLFLOWConfig:
        all_mlflow_config = self.config.model_logging_with_mlflow
        all_mlflow_params = self.params.prepare_base_model
        
        mlflow_config = ModelLoggingWithMLFLOWConfig(
            trained_model_path=Path(all_mlflow_config.trained_model_path),
            scores_file_path=Path(all_mlflow_config.scores_file_path),
            #all_params = self.params.prepare_base_model,
            all_params = all_mlflow_params,
            mlflow_url=all_mlflow_config.mlflow_url
        )

        return mlflow_config

In [4]:
# UPDATE THE COMPONENTS - THIS IS WHERE YOU CREATE THE FILE FOR EVALUATING THE MODEL

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
import json
from urllib.parse import urlparse
from Kidney_classifier import logger

In [9]:
class MLFLOWLogger:
    def __init__(self, config: ModelLoggingWithMLFLOWConfig):
        self.config = config

    def load_evaluation_scores(self):
        with open(self.config.scores_file_path, "r") as file:
            return json.load(file)
        

    def load_trained_model(self):
        return tf.keras.models.load_model(self.config.trained_model_path)
    
    def run_mlflow_log(self):
        scores = self.load_evaluation_scores()
        model = self.load_trained_model()

        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)


            if tracking_url_type_store != "file":
                mlflow.keras.load_model(
                    model, "model", registered_model_name="KIDNEY_DISEASE_CLASSIFIER_FROM_VGG16Model", keras_model_kwargs={"save_format": "h5"}
                )

            else:
                mlflow.keras.log_model(model, "model", keras_model_kwargs={"save_format": "h5"})

In [10]:
# UPDATE THE PIPELINE

try:
    config = ConfigurationManager()
    mlflow_logger_config = config.get_mlflow_config()
    mlflow_logger = MLFLOWLogger(config=mlflow_logger_config)
    mlflow_logger.run_mlflow_log()

except Exception as e:
    raise e

[2025-08-13 18:57:54,343: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-13 18:57:54,348: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-13 18:57:54,350: INFO: common: created directory at: artifacts]
[2025-08-13 18:57:54,625: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


2025/08/13 18:57:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-13 18:57:54,722: WARNING: saving_api: The `save_format` argument is deprecated in Keras 3. We recommend removing this argument as it can be inferred from the file path. Received: save_format=h5]
[2025-08-13 18:57:54,723: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


2025/08/13 18:58:14 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\OLANRE~1\AppData\Local\Temp\tmp89bv275o\model, flavor: tensorflow), fall back to return ['tensorflow==2.17.1']. Set logging level to DEBUG to see the full traceback.
c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\KIDNEY_CLASS_VENV\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
